In [ ]:
!pip install tensorflow
!pip install --upgrade tensorflow-gpu


In [14]:
torch.cuda.empty_cache()
import gc
gc.collect()

149

In [1]:
import os
import zipfile
import time
import copy
import cv2

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import librosa.display
import librosa

import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score

from google.colab import drive


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')

In [11]:
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class MelSpecDataset(Dataset):
    def __init__(self, directory, class_mapping, transform):
        self.directory = directory
        self.class_mapping = class_mapping
        self.data = []
        self.class_data = {}
        self.transform = transform

        for class_name in os.listdir(directory):
            class_dir = os.path.join(directory, class_name)
            self.class_data[class_name] = 0
            if not os.path.isdir(class_dir):
                continue
            class_label = self.class_mapping[class_name]  # Map class name to numerical label
            for npz_file in os.listdir(class_dir):
                npz_path = os.path.join(class_dir, npz_file)
                self.data.append((npz_path, class_label))
                self.class_data[class_name] += 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        npz_path, class_label = self.data[idx]
        mel_spec = np.load(npz_path)['mel_spec']  # Assuming 'mel_spec' is the key for the mel spectrogram array
        mel_spec = self.transform(mel_spec)
        return mel_spec, class_label - 1

# Define the mapping from class names to class indices
class_mapping = {
    'car_horn': 1,
    'dog_barking': 2,
    'drilling': 3,
    'Fart': 4,
    'Guitar': 5,
    'Gunshot_and_gunfire': 6,
    'Hi-hat': 7,
    'Knock': 8,
    'Laughter': 9,
    'Shatter': 10,
    'siren': 11,
    'Snare_drum': 12,
    'Splash_and_splatter': 13
}

# Define the directories
train_directory = "/content/drive/My Drive/DLproject-Numpy/train"
val_directory = "/content/drive/My Drive/DLproject-Numpy/val"

# Create datasets
train_dataset = MelSpecDataset(train_directory, class_mapping, transform)
val_dataset = MelSpecDataset(val_directory, class_mapping, transform)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
datasets = {"train": train_dataset, "val": val_dataset}
dataloaders = {"train": train_dataloader, "val": val_dataloader}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

In [13]:
train_dataset.class_data, sum(train_dataset.class_data.values())

({'Fart': 1000,
  'Guitar': 1000,
  'Gunshot_and_gunfire': 1000,
  'Hi-hat': 1000,
  'Knock': 1000,
  'Laughter': 1000,
  'Shatter': 1000,
  'Snare_drum': 1000,
  'Splash_and_splatter': 1000,
  'car_horn': 1000,
  'dog_barking': 1000,
  'drilling': 1000,
  'siren': 1000},
 13000)

In [7]:
dataset_sizes

{'train': 13000, 'val': 1209}

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CNNModel, self).__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate input size for fully connected layers
        self.fc_input_size = self._calculate_fc_input_size()

        self.fc1 = nn.Linear(self.fc_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def _calculate_fc_input_size(self):
        # Calculate the size of the flattened output after convolution and pooling
        with torch.no_grad():
            x = torch.zeros(1, 3, self.input_shape[0], self.input_shape[1])  # Create dummy input tensor
            x = self.pool(nn.functional.relu(self.conv1(x)))
            x = self.pool(nn.functional.relu(self.conv2(x)))
            x = self.pool(nn.functional.relu(self.conv3(x)))
            return x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, self.fc_input_size)  # Flatten the output for fully connected layers
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class CNNModel2(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(CNNModel2, self).__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)

        # Calculate input size for fully connected layers
        self.fc_input_size = self._calculate_fc_input_size()

        self.fc1 = nn.Linear(self.fc_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def _calculate_fc_input_size(self):
        # Calculate the size of the flattened output after convolution and pooling
        with torch.no_grad():
            x = torch.zeros(1, 3, self.input_shape[0], self.input_shape[1])  # Create dummy input tensor
            x = self.pool(nn.functional.relu(self.conv1(x)))
            x = self.pool(nn.functional.relu(self.conv2(x)))
            x = self.pool(nn.functional.relu(self.conv3(x)))
            return x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, self.fc_input_size)  # Flatten the output for fully connected layers
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:

class VGG10(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(VGG10, self).__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Calculate input size for fully connected layers
        self.fc_input_size = self._calculate_fc_input_size()

        self.classifier = nn.Sequential(
            nn.Linear(self.fc_input_size, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes)
        )

    def _calculate_fc_input_size(self):
        # Calculate the size of the flattened output after convolution and pooling
        with torch.no_grad():
            x = torch.zeros(1, 3, self.input_shape[0], self.input_shape[1])  # Create dummy input tensor
            x = self.features(x)
            return x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, self.fc_input_size)  # Flatten the output for fully connected layers
        x = self.classifier(x)
        return x


In [5]:
class ResNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNetCNN(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(ResNetCNN, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(ResNetBlock, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(ResNetBlock, 128, blocks=2, stride=2)
        self.layer3 = self._make_layer(ResNetBlock, 256, blocks=2, stride=2)
        self.layer4 = self._make_layer(ResNetBlock, 512, blocks=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Calculate input size for fully connected layers
        self.fc_input_size = self._calculate_fc_input_size(input_shape)

        self.fc = nn.Linear(self.fc_input_size, num_classes)

    def _calculate_fc_input_size(self, input_shape):
        # Calculate the size of the flattened output after convolution and pooling
        with torch.no_grad():
            x = torch.zeros(1, 3, input_shape[0], input_shape[1])  # Create dummy input tensor
            x = self.pool(nn.functional.relu(self.conv1(x)))
            x = self.pool(nn.functional.relu(self.layer1(x)))
            x = self.pool(nn.functional.relu(self.layer2(x)))
            x = self.pool(nn.functional.relu(self.layer3(x)))
            x = self.pool(nn.functional.relu(self.layer4(x)))
            return x.view(1, -1).shape[1]

    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [ ]:
class AudioAlexNet(nn.Module):
    def __init__(self, input_shape, num_classes):
        super(AudioAlexNet, self).__init__()
        self.input_shape = input_shape

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        # Calculate the size of the output features after convolutional layers
        self._calculate_fc_input_size()

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.fc_input_size, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def _calculate_fc_input_size(self):
        # Calculate the size of the flattened output after convolution and pooling
        with torch.no_grad():
            x = torch.zeros(1, 3, self.input_shape[0], self.input_shape[1])
            x = self.features(x)
            self.fc_input_size = x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [15]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        print("HELLO")

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            index = 0
            print("STARTING ITERATION")
            for inputs, labels in dataloaders[phase]:

              inputs = inputs.to(device)
              labels = labels.to(device)

              print("BATCH NUMBER = ", index)
              index += 1


              optimizer.zero_grad()
              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

              # print("ACCURACY SO FAR: ", running_corrects.double() / dataset_sizes[phase])

            if phase == 'train':
              print("STEPPING SCEHEDULER")
              scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'EPOCH: {epoch} {phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model weights for the model which has the highest acc.
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [16]:
# Define input shape and number of classes
input_shape = (128, 345, 3)
num_classes = 13  # Assuming 14 output classes

# Instantiate the model
model_ft = ResNetCNN(input_shape, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.9)

In [17]:
# GPU
model_ft = model_ft.to(device)

In [ ]:
# [RESNETCNN] AUGMENTED [& ENSURED VAL HAS BEEN PREPROCESSED]
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
HELLO
STARTING ITERATION
BATCH NUMBER =  0
BATCH NUMBER =  1
BATCH NUMBER =  2
BATCH NUMBER =  3
BATCH NUMBER =  4
BATCH NUMBER =  5
BATCH NUMBER =  6
BATCH NUMBER =  7
BATCH NUMBER =  8
BATCH NUMBER =  9
BATCH NUMBER =  10
BATCH NUMBER =  11
BATCH NUMBER =  12
BATCH NUMBER =  13
BATCH NUMBER =  14
BATCH NUMBER =  15
BATCH NUMBER =  16
BATCH NUMBER =  17
BATCH NUMBER =  18
BATCH NUMBER =  19
BATCH NUMBER =  20
BATCH NUMBER =  21
BATCH NUMBER =  22
BATCH NUMBER =  23
BATCH NUMBER =  24
BATCH NUMBER =  25
BATCH NUMBER =  26
BATCH NUMBER =  27
BATCH NUMBER =  28
BATCH NUMBER =  29
BATCH NUMBER =  30
BATCH NUMBER =  31
BATCH NUMBER =  32
BATCH NUMBER =  33
BATCH NUMBER =  34
BATCH NUMBER =  35
BATCH NUMBER =  36
BATCH NUMBER =  37
BATCH NUMBER =  38
BATCH NUMBER =  39
BATCH NUMBER =  40
BATCH NUMBER =  41
BATCH NUMBER =  42
BATCH NUMBER =  43
BATCH NUMBER =  44
BATCH NUMBER =  45
BATCH NUMBER =  46
BATCH NUMBER =  47
BATCH NUMBER =  48
BATCH NUMBER =  49
BATCH NUMBER =

In [ ]:
# SAVING RESNET CNN
def save_model(model, model_name):
  torch.save(model.state_dict(), f'{model_name}_weights.pth')
  torch.save(model, f'{model_name}.pth')

save_model(model_ft, "RESNETCNN")



In [ ]:
from google.colab import files
files.download('CNNModel_weights.pth')
files.download('CNNModel.pth')


In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
HELLO
STARTING ITERATION
BATCH NUMBER =  0
BATCH NUMBER =  1
BATCH NUMBER =  2
BATCH NUMBER =  3
BATCH NUMBER =  4
BATCH NUMBER =  5
BATCH NUMBER =  6
BATCH NUMBER =  7
BATCH NUMBER =  8
BATCH NUMBER =  9
BATCH NUMBER =  10
BATCH NUMBER =  11
BATCH NUMBER =  12
BATCH NUMBER =  13
BATCH NUMBER =  14
BATCH NUMBER =  15
BATCH NUMBER =  16
BATCH NUMBER =  17
BATCH NUMBER =  18
BATCH NUMBER =  19
BATCH NUMBER =  20
BATCH NUMBER =  21
BATCH NUMBER =  22
BATCH NUMBER =  23
BATCH NUMBER =  24
BATCH NUMBER =  25
BATCH NUMBER =  26
BATCH NUMBER =  27
BATCH NUMBER =  28
BATCH NUMBER =  29
BATCH NUMBER =  30
BATCH NUMBER =  31
BATCH NUMBER =  32
BATCH NUMBER =  33
BATCH NUMBER =  34
BATCH NUMBER =  35
BATCH NUMBER =  36
BATCH NUMBER =  37
BATCH NUMBER =  38
BATCH NUMBER =  39
BATCH NUMBER =  40
BATCH NUMBER =  41
BATCH NUMBER =  42
BATCH NUMBER =  43
BATCH NUMBER =  44
BATCH NUMBER =  45
BATCH NUMBER =  46
BATCH NUMBER =  47
BATCH NUMBER =  48
BATCH NUMBER =  49
BATCH NUMBER =

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

Epoch 0/1
----------
HELLO
STARTING ITERATION
BATCH NUMBER =  0
BATCH NUMBER =  1
BATCH NUMBER =  2
BATCH NUMBER =  3
BATCH NUMBER =  4
BATCH NUMBER =  5
BATCH NUMBER =  6
BATCH NUMBER =  7
BATCH NUMBER =  8
BATCH NUMBER =  9
BATCH NUMBER =  10
BATCH NUMBER =  11
BATCH NUMBER =  12
BATCH NUMBER =  13
BATCH NUMBER =  14
BATCH NUMBER =  15
BATCH NUMBER =  16
BATCH NUMBER =  17
BATCH NUMBER =  18
BATCH NUMBER =  19
BATCH NUMBER =  20
BATCH NUMBER =  21
BATCH NUMBER =  22
BATCH NUMBER =  23
BATCH NUMBER =  24
BATCH NUMBER =  25
BATCH NUMBER =  26
BATCH NUMBER =  27
BATCH NUMBER =  28
BATCH NUMBER =  29
BATCH NUMBER =  30
BATCH NUMBER =  31
BATCH NUMBER =  32
BATCH NUMBER =  33
BATCH NUMBER =  34
BATCH NUMBER =  35
BATCH NUMBER =  36
BATCH NUMBER =  37
BATCH NUMBER =  38
BATCH NUMBER =  39
BATCH NUMBER =  40
BATCH NUMBER =  41
BATCH NUMBER =  42
BATCH NUMBER =  43
BATCH NUMBER =  44
BATCH NUMBER =  45
BATCH NUMBER =  46
BATCH NUMBER =  47
BATCH NUMBER =  48
BATCH NUMBER =  49
BATCH NUMBER =

In [ ]:

class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 64 * 86, 128)  # Adjust input size based on your input dimensions
        self.fc2 = nn.Linear(128, 13)  # 13 is the number of output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

inputs = inputs.squeeze(-1)  # Remove the last dimension
inputs = inputs.squeeze(1)   # Remove the first dimension


# Convert inputs to float
inputs = inputs.float()

# Convert bias tensor to the same data type as inputs
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.bias.data = layer.bias.data.float()


# Define your model
model = YourModel()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the number of epochs
num_epochs = 10

# Move the model to the appropriate device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        # Pass inputs through the model with explicit data type conversion
        outputs = model(inputs.float())

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    # Print training/validation statistics
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss/len(train_dataloader.dataset):.4f}, "
          f"Train Accuracy: {100*correct_train/total_train:.2f}%, "
          f"Val Loss: {val_loss/len(val_dataloader.dataset):.4f}, "
          f"Val Accuracy: {100*correct_val/total_val:.2f}%")


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [64, 128, 345, 3, 1]